<p align="center">
  <img src="https://huggingface.co/speakleash/Bielik-7B-Instruct-v0.1/raw/main/speakleash_cyfronet.png">
</p>

W tym notebooku przejdziemy całą ścieżkę - od uruchomienia modelu aż po zwrot danych w formacie JSON. Zaczynajmy! :)

### Przygotowanie środowiska

In [1]:
#%%capture

!pip install accelerate -q
!pip install -i https://pypi.org/simple/ bitsandbytes -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 7.0 MB/s eta 0:00:00


In [2]:
import gc
import torch

In [3]:
import warnings
warnings.filterwarnings("ignore") # tylko na potrzeby naszego demo ;)

### Inicjalizacja modelu (4 bit)

In [4]:
from transformers import (AutoModelForCausalLM, AutoTokenizer,
                          BitsAndBytesConfig, TextStreamer)

# Wybór urządzenia do uruchomienia modelu
device = "cuda"
model_name = 'speakleash/Bielik-7B-Instruct-v0.1'

# Ustawienie kwantyzacji na 4 bity, aby zmieścić na GPU na Colabie
load_4_bit = True

# Ustawienie tokenizera i kluczowych tokenów
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Obsługa streamingu
streamer = TextStreamer(tokenizer,
                        skip_prompt=True,
                        skip_special_tokens=True)

# Przygotowanie zmiennej pod konfigurację kwantyzacji
quantization_config = None

if load_4_bit:
  quantization_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_compute_dtype=torch.bfloat16)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    quantization_config=quantization_config)



tokenizer_config.json:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/593 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

### Funkcja pomocnicza do generowania odpowiedzi

In [5]:
# Parametry modelu
temperature = 0.0
max_tokens = 256
top_k = 100
top_p = 1

def generate(prompt: str,
             max_tokens: int = 256,
             temperature: float = 0.0,
             system: str = None,
             return_str: bool = False):

  messages = []

  # Przygotowanie template z promptem
  if system:
    messages.append({"role": "system",
                     "content": system})

  messages.append({"role": "user",
                   "content": prompt})

  input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt")
  model_inputs = input_ids

  # Przerzucenie inputu na GPU, jeśli jest dostępne
  if torch.cuda.is_available():
    model_inputs = input_ids.to(device)

  # Generowanie odpowiedzi przez model
  outputs = model.generate(
      model_inputs,
      streamer = streamer,
      max_new_tokens=max_tokens,
      do_sample=True if temperature else False,
      temperature = temperature,
      top_k = top_k,
      top_p = top_p)

  # Wydrukowanie odpowiedzi modelu bądź zwrotka do zmiennej
  if return_str:
    return tokenizer.batch_decode(outputs, skip_special_tokens=False)

#### Test

In [6]:
# Test generowania tekstu przez model
generate("Hej, jesteś tam?")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Cześć! Tak, jestem tutaj. W czym mogę Ci pomóc?


In [7]:
# Test z promptem systemowym i właściwym
generate(prompt="Hej, czym się zajmujesz?",
         system="Jesteś programistą baz danych, który chce rozwijać swoje kompetencje w obszarze AI")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Cześć! Jako programista baz danych specjalizuję się w projektowaniu, tworzeniu i zarządzaniu systemami, które przechowują i organizują duże zbiory danych. W ostatnim czasie zacząłem interesować się sztuczną inteligencją i uczeniem maszynowym, aby poszerzyć swoje kompetencje i zrozumieć, jak te technologie mogą wpływać na moją dziedzinę. Uczestniczę w kursach i czytam artykuły na temat AI, aby zdobyć wiedzę i umiejętności związane z wykorzystaniem AI w bazach danych. Chciałbym w przyszłości wdrażać rozwiązania AI i machine learning w mojej pracy, aby pomagać organizacjom w lepszym zrozumieniu i analizie ich danych. Czy masz jakieś pytania


# Przykład 1

Bielik może nadawać się także do analizy sentymentu danej wiadomości, artykułu czy też komentarzy. Sprawdźmy, jak nasz model sobie z tym poradzi!

Wybrałem pierwszy z brzegu artykuł: https://wydarzenia.interia.pl/kraj/news-ekstremalne-warunki-niemal-w-calym-kraju-sypia-sie-alerty,nId,7763701

Najpierw musimy wydobyć tekst z artykułu! Do tego przydadzą się pythonowe biblioteki **requests** oraz **justext**!

In [8]:
# Instalacja niezbędnych bibliotek
!pip install justext

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.8/837.8 kB 12.4 MB/s eta 0:00:00


In [9]:
import requests
import justext

url = "https://wydarzenia.interia.pl/kraj/news-ekstremalne-warunki-niemal-w-calym-kraju-sypia-sie-alerty,nId,7763701"

def get_article_from_url(url: str):
  # Przygotowanie zmiennej do zbierania fragmentów artykułu ze strony internetowej
  article_parts = []

  # Zebranie odpowiedzi w HTML
  response = requests.get(url)

  # Przetworzenie odpowiedzi przy użyciu biblioteki justext
  paragraphs = justext.justext(response.content, justext.get_stoplist("Polish"))
  for paragraph in paragraphs:
    if not paragraph.is_boilerplate:
      article_parts.append(paragraph.text)

  return " ".join(article_parts)

In [10]:
# Test działania funkcji
get_article_from_url(url)

'Ekstremalne warunki niemal w całym kraju. Sypią się alerty Pogoda w najbliższym czasie nie da odetchnąć. Niedziela będzie kolejnym dniem z tropikalnymi upałami. W wielu miejscach termometry wskażą powyżej 30 stopni Celsjusza. W związku z ekstremalnymi warunkami IMGW wydał ostrzeżenia dla ponad połowy kraju. W lasach obowiązuje bardzo wysokie zagrożenie pożarowe. Fala upałów w Polsce. IMGW wydał ostrzeżenia/Zbyszek Kaczmarek/Agencja FORUM Reklama W niedzielę z nieba znów poleje się żar. Niemal wszędzie notowane będą ponad 30-stopniowe upały. W związku z ekstremalnymi warunkami IMGW wydał ostrzeżenia pierwszego i drugiego stopnia dla połowy kraju. Fala upałów w Polsce. IMGW wydał ostrzeżenia dla połowy kraju W ciągu dnia na wschodzie i południu Polski niebo będzie bezchmurne. Na pozostałym obszarze może wystąpić przelotny deszcz, a lokalnie - także burze. Wiatr, szczególnie nad morzem będzie dość silny. Porywy mogą sięgać do 65 km/h. Reklama Będzie upalnie. Niemal w całej Polsce słupki 

In [11]:
# Pobranie treści artykułu do zmiennej
article = get_article_from_url(url)

# Przygotowanie promptów
system = 'Jesteś ekspertem od analizy wydźwięku. Zapoznaj się z treścią podanego artykułu i podaj jaki jest wydźwięk oraz sentyment tego artykułu.'
prompt = f"Tekst do analizy: {article}"

# Zwiększenie limitu tokenów
max_tokens = 500

# Generowanie odpowiedzi Bielika!
generate(prompt,
         max_tokens=max_tokens)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Tekst do analizy:
Ekstremalne warunki niemal w całym kraju. Sypią się alerty Pogoda w najbliższym czasie nie da odetchnąć. Niedziela będzie kolejnym dniem z tropikalnymi upałami. W wielu miejscach termometry wskażą powyżej 30 stopni Celsjusza. W związku z ekstremalnymi warunkami IMGW wydał ostrzeżenia pierwszego i drugiego stopnia dla połowy kraju. Fala upałów w Polsce. IMGW wydał ostrzeżenia dla połowy kraju W ciągu dnia na wschodzie i południu Polski niebo będzie bezchmurne. Na pozostałym obszarze może wystąpić przelotny deszcz, a lokalnie - także burze. Wiatr, szczególnie nad morzem będzie dość silny. Porywy mogą sięgać do 65 km/h. Będzie upalnie. Niemal w całej Polsce słupki rtęci powędrują w okolicę 30 stopni Celsjusza, a w wielu miejscach przekroczą tę granicę. Najcieplej będzie w Lublinie, Rzeszowie i Warszawie - tam termometry wskażą nawet 34 kreski. IMGW wydał ostrzeżenia drugiego stopnia dla centralnej, południowej i wschodniej części kraju. Alerty obowiązują w województwach

### Odpowiedź (na ten moment z Areny)

Po dokładnej analizie tekstu, wydźwięk i sentyment tego artykułu można określić jako:

Wydźwięk: Neutralny z tendencją do negatywnego Sentyment: Lekko negatywny

Uzasadnienie:

1. Neutralny wydźwięk:

* Artykuł przekazuje głównie informacje o prognozach pogody i ostrzeżeniach IMGW.
* Nie zawiera wyraźnie pozytywnych lub negatywnych opinii.

2. Tendencja do negatywnego wydźwięku:

* Opisuje ekstremalne warunki pogodowe, które mogą być uciążliwe dla ludzi.
* Wskazuje na zagrożenia związane z upałami, takie jak susza i ryzyko pożarów.

3. Lekko negatywny sentyment:

* Używa słów takich jak "ekstremalne warunki", "ostrzeżenia", "zagrożenie pożarowe", co ma negatywne konotacje.
* Informuje o nieprzyjemnych doświadczeniach związanych z upałami (np. "z nieba znów poleje się żar").
4. Brak wyraźnie pozytywnych elementów:

* Nie ma wzmianek o potencjalnych pozytywnych aspektach upałów (np. możliwości korzystania z plaży czy basenu).

5. Ostrzeżenia i zalecenia:

* Artykuł zawiera wiele ostrzeżeń i zaleceń bezpieczeństwa, co może wywoływać poczucie niepokoju u czytelników.

Podsumowując, artykuł ma głównie charakter informacyjny, ale ze względu na opisywane ekstremalne warunki pogodowe i związane z nimi zagrożenia, jego sentyment jest lekko negatywny.

# Przykład 2

Sprawdźmy ten sam prompt (czyli treść zadania dla modelu), ale tym razem użyjmy innego materiału źródłowego.

In [13]:
article = "Wasze odestki są wręcz cudowne. Gdyby były niższe byłoby mnie stać na masło, a tak to tylko margaryna i tynk ze ściany na śniadanie."

system = 'Jesteś ekspertem od analizy wydźwięku. Zapoznaj się z treścią podanego artykułu i podaj jaki jest wydźwięk oraz sentyment tego tekstu.'
prompt = f"Tekst do analizy: {article}"
max_tokens = 550

generate(prompt,
         max_tokens=max_tokens)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 W tym tekście autorka wyraża swoje zadowolenie z odsetek, które otrzymuje. Mówi, że gdyby odsetki były niższe, byłaby w stanie pozwolić sobie na zakup masła. Obecnie, z powodu wyższych odsetek, może sobie pozwolić jedynie na margarynę i tynk ze ściany na śniadanie. Autorka żartobliwie przedstawia swoją sytuację finansową, ale jednocześnie podkreśla, jak ważne są dla niej odsetki.


### Odpowiedź Bielika (z Areny)

Po dokładnej analizie podanego tekstu, można stwierdzić, że wydźwięk i sentyment tego artykułu są mieszane, z przewagą negatywnego sentymentu.

Wydźwięk:

* Pozytywny: Autor wyraża pozytywny stosunek do "odestków" (prawdopodobnie chodzi o odsetki), nazywając je "cudownymi".
* Negatywny: Autor sugeruje, że wysokie odsetki powodują problemy finansowe, uniemożliwiając mu kupno podstawowych produktów spożywczych.

Sentyment:

* Pozytywny: Komplementowanie "odestków" jako "cudownych" może być interpretowane jako pozytywny sentyment wobec tego konkretnego aspektu.
* Negatywny: Opis trudności finansowych i braku możliwości zakupu podstawowych produktów spożywczych wyraża negatywny sentyment wobec sytuacji finansowej autora i potencjalnie wobec wysokich odsetek.

Podsumowując, wydźwięk jest mieszany, ale sentyment jest głównie negatywny, ponieważ autor wyraża niezadowolenie z wpływu wysokich odsetek na jego sytuację finansową.

# Przykład 3

Teraz Bielik zostanie poproszony o nieco bardziej ustrukturyzowaną odpowiedź. Zobaczcie sami...

In [18]:
article = get_article_from_url(url)

system = 'Jesteś ekspertem od analizy wydźwięku. Zapoznaj się z treścią podanego artykułu i podaj jaki jest wydźwięk oraz sentyment tego tekstu. Wyniki zwróć w formacie obiektu JSON.'
prompt = f"Tekst do analizy: {article}"
max_tokens = 550

generate(prompt,
         max_tokens=max_tokens)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Analiza:

Mecz piłki nożnej: FC Barcelona vs Athletic Club
Wynik: FC Barcelona 2:1 Athletic Club
Data: 20 sierpnia 2022
Trener FC Barcelona: Hansi Flick
Najlepszy strzelec FC Barcelona w tym sezonie: Robert Lewandowski (1 goal)
Podsumowanie:
Robert Lewandowski strzelił gola w meczu FC Barcelona z Athletic Bilbao, który zakończył się zwycięstwem "Blaugrany" 2:1. Polak otworzył wynik spotkania, ale wcześniej zmarnował kilka dogodnych okazji.
Przebieg meczu:
Pierwsza połowa była wyrównana, z niewielką przewagą FC Barcelona. Gospodarze objęli prowadzenie w 24. minucie po bramce Lamine Yamala, który wykorzystał rzut wolny i błąd obrońcy Athletic Bilbao. Robert Lewandowski miał dwie dobre okazje do zdobycia gola, ale nie zdołał pokonać bramkarza rywali.
W drugiej połowie obraz gry nie uległ zmianie. FC Barcelona dominowała, ale Athletic Bilbao również miało swoje szanse. W 57. minucie Oihan Sancet wykorzystał rzut karny podyktowany za faul na Alexie Berenguera i doprowadził do remisu.
W 70.

Na podstawie analizy wydźwięku i sentymentu podanego tekstu, oto wyniki w formacie obiektu JSON:

```json
{
  "Wydźwięk": "Negatywny",
  "Sentyment": "Neutralny",
  "Uzasadnienie": "Tekst opisuje ekstremalne warunki pogodowe, takie jak upały i zagrożenie pożarowe, co ma negatywny wydźwięk. Jednakże, ton artykułu jest głównie informacyjny i ostrzegawczy, co nadaje mu neutralny sentyment."
}
```

Wyjaśnienie krok po kroku:

1. Wydźwięk: Negatywny

* Tekst opisuje ekstremalne warunki pogodowe, takie jak upały powyżej 30 stopni Celsjusza, ostrzeżenia IMGW, zagrożenie pożarowe w lasach i suszę hydrologiczną.
* Te zjawiska pogodowe są zazwyczaj postrzegane jako niekorzystne lub szkodliwe, co nadaje tekstowi negatywny wydźwięk.

2. Sentyment: Neutralny

* Tekst ma głównie charakter informacyjny i ostrzegawczy.
* Autor nie wyraża osobistych emocji ani opinii wobec opisywanych zjawisk.
* Skupia się na przekazaniu faktów i ostrzeżeń, co nadaje tekstowi neutralny sentyment.

3. Uzasadnienie:

* Negatywny wydźwięk wynika z opisu niekorzystnych warunków pogodowych i zagrożeń z nimi związanych.
* Neutralny sentyment wynika z obiektywnego, informacyjnego tonu artykułu, bez wyraźnego emocjonalnego zaangażowania autora.

Sami oceńcie, jak Bielik poradził sobie z postawionymi przed nim zadaniami! :)